### Import Library dan import dataset

In [1]:
import pulp
import pandas as pd
import psutil
import time
import os

In [2]:
room_excel = pd.read_excel("Dataset/Gedung TULT.xlsx", usecols=['Nama Ruangan'])
room_list = room_excel['Nama Ruangan'].tolist()
room_list

['TULT-0601',
 'TULT-0602',
 'TULT-0603',
 'TULT-0701',
 'TULT-0702',
 'TULT-0703',
 'TULT-0706',
 'TULT-0707',
 'TULT-0709',
 'TULT-0710',
 'TULT-0711',
 'TULT-0714',
 'TULT-0715',
 'TULT-0716',
 'TULT-1507',
 'TULT-1509',
 'TULT-1510']

In [3]:
class_excel = pd.read_excel("Dataset/Kelas FIF.xlsx", sheet_name="Kelas FIF", usecols=['Kelas FIF'])
class_list = class_excel['Kelas FIF'].tolist()
class_list

['IF-47-01',
 'IF-47-02',
 'IF-47-03',
 'IF-47-04',
 'IF-47-05',
 'IF-47-06',
 'IF-47-07',
 'IF-47-08',
 'IF-47-09',
 'IF-47-10',
 'IF-47-11',
 'IF-47-12',
 'IF-47-INT',
 'IT-47-01',
 'IT-47-02',
 'IT-47-03',
 'IT-47-04',
 'SE-47-01',
 'SE-47-02',
 'SE-47-03',
 'SE-47-04',
 'DS-47-01',
 'DS-47-02',
 'DS-47-03']

In [4]:
subject_excel = pd.read_excel("Dataset/Mata Kuliah FIF.xlsx", sheet_name="FIF Semester 1", usecols=['Kode'])
subject_list = subject_excel['Kode'].tolist()
subject_list

['UKJXB2',
 'UAJXA2',
 'CII1E3',
 'CII1A3',
 'CII1C2',
 'CII1B3',
 'CII1D3 ',
 'CTJ1A2',
 'CRI1A2',
 'CRI1B3',
 'CSI1A2']

In [5]:
# Dictionary mapping each course to its number of consecutive timeslots
# course_consecutive_timeslots = {
#     'UKJXB2': 2,
#     'UAJXA2': 3,
#     'CII1E3': 3,
#     'CII1A3': 3,
#     'CII1C2': 2,
#     'CII1B3': 3,
#     'CII1D3': 3,
#     'CTJ1A2': 2,
#     'CRI1A2': 2,
#     'CRI1B3': 3,
#     'CSI1A2': 2
# }

In [6]:
# Read the dataset and filter by status
dosen_dataset = pd.read_excel("Dataset/data dosen.xlsx", sheet_name="all dosen", usecols=['Kode', 'Status'])

### Parameter

In [7]:
rooms = room_list
students_groups = class_list
subjects = subject_list
permanent_lectures = dosen_dataset.loc[dosen_dataset['Status'] == 1, 'Kode'].tolist()
non_permanent_lectures = dosen_dataset.loc[dosen_dataset['Status'] == 0, 'Kode'].tolist()
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
timeslots = ['timeslot1', 'timeslot2', 'timeslot3', 'timeslot4', 'timeslot5', 'timeslot6', 'timeslot7', 'timeslot8', 'timeslot9', 'timeslot10', 'timeslot11', 'timeslot12']

### Decision Variable

In [8]:
x = pulp.LpVariable.dicts("x", (rooms, students_groups, timeslots, days, subjects, permanent_lectures + non_permanent_lectures), cat='Binary')

### Objective function

In [9]:
# # Objective Function
lp_problem = pulp.LpProblem("Room Scheduling", pulp.LpMinimize)

# Objective function: minimize the total number of rooms used in timeslot1, timeslot5, and timeslot11
lp_problem += pulp.lpSum(x[classroom][group][time_slot][day][subject][lecturer] for classroom in rooms
                                                           for group in students_groups
                                                           for time_slot in ['timeslot1', 'timeslot5', 'timeslot6', 'timeslot11', 'timeslot12']
                                                           for day in ['Saturday']
                                                           for subject in subjects
                                                           for lecturer in permanent_lectures + non_permanent_lectures)

c:\Users\sltna\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### Constraints

In [10]:
# Monitor resource usage
start_time = time.time()

cpu_usage = []
memory_usage_before = []
memory_usage_after = []
disk_usage_before = []
disk_usage_after = []

def monitor_resource_usage_before():
    cpu_usage.append(psutil.cpu_percent(interval=None))
    memory_usage_before.append(psutil.virtual_memory().used / (1024 ** 2))  # in MB
    disk_usage_before.append(psutil.disk_usage('/').used / (1024 ** 2))  # in MB

def monitor_resource_usage_after():
    cpu_usage.append(psutil.cpu_percent(interval=None))
    memory_usage_after.append(psutil.virtual_memory().used / (1024 ** 2))  # in MB
    disk_usage_after.append(psutil.disk_usage('/').used / (1024 ** 2))  # in MB

In [11]:
# Record resource usage before solving
monitor_resource_usage_before()

In [12]:
# Constraint: One Lecture per Time Slot and Classroom
for classroom in rooms:
    for group in students_groups:
        for time_slot in timeslots:
            for day in days:
                lp_problem += pulp.lpSum(x[classroom][group][time_slot][day][subject][lecturer] 
                                    for subject in subjects 
                                    for lecturer in permanent_lectures + non_permanent_lectures) <= 1

# Constraint: Only permanent lecturers do not teach on timeslot4, timeslot5, timeslot6 on Thursdays
for lecturer in permanent_lectures:
    for classroom in rooms:
        for group in students_groups:
            for subject in subjects:
                if time_slot in ['timeslot4', 'timeslot5', 'timeslot6']:
                    lp_problem += x[classroom][group][subject]['thursday'][time_slot][lecturer] == 0


# Constraint: Each course is scheduled to take consecutive timeslots
# for subject in subjects:
#     last_char = int(subject.strip()[-1])  # Get the last character of the course name as an integer
#     for classroom in rooms:
#         for group in students_groups:
#             for day in days:
#                 # Iterate over consecutive time slots based on the last character of the subject
#                 for i in range(len(timeslots) - last_char + 1):
#                     consecutive_timeslots = timeslots[i:i + last_char]
                    
#                     # Add constraint to ensure only one time slot is chosen for each consecutive set
#                     for lecturer in permanent_lectures + non_permanent_lectures:
#                         lp_problem += pulp.lpSum(x[classroom][group][time_slot][day][subject][lecturer] for time_slot in consecutive_timeslots) <= 1
# for subject in subjects:
#     last_char = int(subject.strip()[-1])  # Get the last character of the course name as an integer
#     for classroom in rooms:
#         for group in students_groups:
#             for day in days:
#                 for lecturer in permanent_lectures + non_permanent_lectures:
#                     for i in range(last_char):
#                         lp_problem += pulp.lpSum(x[classroom][group][timeslot[i + j]][day][subject][lecturer] for j in range(last_char)) <= 1

# [bermasalah] Constraint: Each student attends minimum two courses and a maximum four courses for every day
# for group in students_groups:
#     for day in days:
#         # Create a list of decision variables for each combination of room, timeslot, course, and lecture
#         decision_vars = [x[classroom][group][time_slot][day][subject][lecturer] 
#                          for classroom in rooms 
#                          for time_slot in timeslots 
#                          for subject in subjects 
#                          for lecturer in permanent_lectures + non_permanent_lectures]
        
#         # Add constraint using pulp.lpSum
#         lp_problem += pulp.lpSum(decision_vars) <= 4

# [bermasalah] Constraint to ensure no courses in timeslot 5 and timeslot 6 on Friday
for classroom in rooms:
    for group in students_groups:
        for timeslot in ['timeslot5', 'timeslot6']:
            for day in ['Friday']:
                for subject in subjects:
                    for lecturer in permanent_lectures + non_permanent_lectures:
                        lp_problem += x[classroom][group][timeslot][day][subject][lecturer] == 0

# Constraint: 1 room can only be used by 1 student group in 1 timeslot
for classroom in rooms:
    for time_slot in timeslots:
        lp_problem += pulp.lpSum(x[classroom][group][time_slot][day][subject][lecturer] for group in students_groups
                                                                          for day in days
                                                                          for subject in subjects
                                                                          for lecturer in permanent_lectures + non_permanent_lectures) <= 1

# Constraint: Lecturers can only teach exactly 1 student group in 1 timeslot
for lecturer in permanent_lectures + non_permanent_lectures:
    for time_slot in timeslots:
        lp_problem += pulp.lpSum(x[classroom][group][time_slot][day][subject][lecturer] for classroom in rooms
                                                                          for group in students_groups
                                                                          for day in days
                                                                          for subject in subjects) <= 1

# Constraint to ensure Rooms TULT-1507, TULT-1509, and TULT-1510 can only be used by IF-47-INT students
for classroom in ['TULT-1507', 'TULT-1509', 'TULT-1510']:
    for subject in subjects:
        for lecturer in permanent_lectures + non_permanent_lectures:
            for group in [group for group in students_groups if group != 'IF-47-INT']:
                for day in days:
                    for time_slot in timeslots:
                        lp_problem += x[classroom][group][time_slot][day][subject][lecturer] == 0



#### Constraint untuk mata kuliah

In [13]:
# Constraint: Each student group is scheduled to take exactly 1 course in the timeslot every day
for group in students_groups:
    for time_slot in timeslots:
        for day in days:
            lp_problem += pulp.lpSum(x[classroom][group][time_slot][day][subject][lecturer] for classroom in rooms
                                                                              for subject in subjects
                                                                              for lecturer in permanent_lectures + non_permanent_lectures) == 1

# Constraint: Each group must take exactly one course per timeslot each day
for group in students_groups:
    for day in days:
        for time_slot in timeslots:
            lp_problem += pulp.lpSum(x[classroom][group][time_slot][day][subject][lecturer] for classroom in rooms for subject in subjects for lecturer in permanent_lectures + non_permanent_lectures) == 1

# Constraint: Courses must be scheduled for consecutive timeslots based on their final character
for subject in subjects:
    try:
        num_consecutive_slots = int(subject[-1])
    except ValueError:
        num_consecutive_slots = 1  # Default to 1 if the last character is not an integer

    for classroom in rooms:
        for group in students_groups:
            for day in days:
                for lecture in permanent_lectures + non_permanent_lectures:
                    for i in range(len(timeslots) - num_consecutive_slots + 1):
                        lp_problem += pulp.lpSum(x[classroom][group][timeslots[i+j]][day][subject][lecturer] for j in range(num_consecutive_slots)) <= 1

# Constraint: Enforce specific groups for specific courses
for subject in subjects:
    for group in students_groups:
        if (subject == 'UAJXA2' and not group.startswith('IF')) or \
           (subject == 'CTJ1A2' and not group.startswith('IT')) or \
           (subject in ['CRI1A2', 'CRI1B3'] and not group.startswith('SE')) or \
           (subject == 'CSI1A2' and not group.startswith('DS')):
            for day in days:
                for time_slot in timeslots:
                    for classroom in rooms:
                        for lecture in permanent_lectures + non_permanent_lectures:
                            lp_problem += x[classroom][group][time_slot][day][subject][lecturer] == 0

# Constraint: Ensure each group learns the specified courses at least once during the week
# specified_courses = ['UKJXB2', 'CII1E3', 'CII1A3', 'CII1C2', 'CII1B3', 'CII1D3']
# for group in students_groups:
#     for subject in specified_courses:
#         lp_problem += (pulp.lpSum(
#                 x[classroom][group][time_slot][day][subject][lecturer]
#                 for classroom in rooms
#                 for day in days
#                 for time_slot in timeslots
#                 for lecturer in permanent_lectures + non_permanent_lectures
#             )
#             >= 1
#         )

# Constraint: Ensure some subject is learned by specific group
# for group in students_groups:
#     for subject in ['CTJ1A2', 'UAJXA2', 'CRI1A2', 'CRI1B3', 'CSI1A2']:
#         for day in days:
#             for time_slot in timeslots:
#                 for classroom in rooms:
#                     for lecturer in permanent_lectures + non_permanent_lectures:
#                         if subject == 'CTJ1A2':
#                             if group not in ['IT-47-01', 'IT-47-02', 'IT-47-03', 'IT-47-04']:
#                                 lp_problem += x[classroom][group][time_slot][day][subject][lecturer] == 0
#                         elif subject == 'UAJXA2':
#                             if group not in ['IF-47-01', 'IF-47-02', 'IF-47-03', 'IF-47-04', 'IF-47-05', 'IF-47-06', 'IF-47-07', 'IF-47-08', 'IF-47-09', 'IF-47-010', 'IF-47-011', 'IF-47-12', 'IF-47-INT']:
#                                 lp_problem += x[classroom][group][time_slot][day][subject][lecturer] == 0
#                         elif subject == 'CRI1A2' and subject == 'CRI1B3':
#                             if group not in ['SE-47-01', 'SE-47-02', 'SE-47-03', 'SE-47-04']:
#                                 lp_problem += x[classroom][group][time_slot][day][subject][lecturer] == 0
#                         elif subject == 'CSI1A2':
#                             if group not in ['DS-47-01', 'DS-47-02', 'DS-47-03']:
#                                 lp_problem += x[classroom][group][time_slot][day][subject][lecturer] == 0
#                         else:
#                             # All other courses can be accessed by all groups
#                             lp_problem += x[classroom][group][time_slot][day][subject][lecturer] >= 0

# Constraint: Ensure CTJ1A2 and UAJXA2 are studied once a week by the allowed groups
# allowed_courses = {'CTJ1A2': ['group1'], 'UAJXA2': ['group4', 'group5']}
# for course, groups in allowed_courses.items():
#     for group in groups:
#         lp_problem += (
#             pulp.lpSum(
#                 x[classroom][group][time_slot][day][subject][lecturer]
#                 for classroom in rooms
#                 for day in days
#                 for time_slot in timeslots
#                 for lecturer in permanent_lectures + non_permanent_lectures
#             )
#             == 1
#         )

# Constraint: Ensure the subject uses 2 timeslots consecutively in one day and once a week
# for subject in ['CTJ1A2']:
#     if subject == 'CTJ1A2':
#         for group in ['IT-47-01', 'IT-47-02', 'IT-47-03', 'IT-47-04']:
#             for day in days:
#                 # Create a list of decision variables for the subject, group, and day
#                 decision_vars = [x[classroom][group][time_slot][day][subject][lecturer]
#                                  for classroom in rooms
#                                  for time_slot in timeslots
#                                  for lecturer in permanent_lectures + non_permanent_lectures
#                                  if (classroom, time_slot, day) in x.keys()]
                
#                 # Add constraint: Subject uses 2 timeslots consecutively in one day
#                 for i in range(len(timeslots) - 1):
#                     lp_problem += pulp.lpSum(decision_vars[i:i+2]) <= 1
                
#                 # Add constraint: Subject is scheduled once a week
#                 lp_problem += pulp.lpSum(decision_vars) == 1 # CTJ1A2 is studied only once a week by each group

# # Constraint: Ensure CTJ1A2 course is studied by group 4 and group 5 and is only studied once a week
# for subject in subjects:
#     if subject == 'UAJXA2':
#         for group in ['IF-47-01', 'IF-47-02', 'IF-47-03', 'IF-47-04', 'IF-47-05', 'IF-47-06', 'IF-47-07', 'IF-47-08', 'IF-47-09', 'IF-47-010', 'IF-47-011', 'IF-47-12', 'IF-47-INT']:
#             for day in days:
#                 # Create a list of decision variables for the subject, group, and day
#                 decision_vars = [x[classroom][group][time_slot][day][subject][lecturer]
#                                  for classroom in rooms
#                                  for time_slot in timeslots
#                                  for lecturer in permanent_lectures + non_permanent_lectures
#                                  if (classroom, time_slot, day) in x.keys()]
                
#                 # Add constraint using pulp.lpSum
#                 lp_problem += pulp.lpSum(decision_vars) == 1  # CTJ1A2 is studied only once a week by each group

# [bermasalah] Constraint: Ensure subjects UKJXB2, CII1E3, CII1A3, CII1C2, CII1B3, CII1D3 are studied by all groups and are scheduled according to the timeslot used
# for subject in ['UKJXB2', 'CII1E3', 'CII1A3', 'CII1C2', 'CII1B3', 'CII1D3']:
#     for group in students_groups:
#         for day in days:
#             # Create a list of decision variables for the subject, group, and day
#             decision_vars = [x[classroom][group][time_slot][day][subject][lecturer]
#                              for classroom in rooms
#                              for time_slot in timeslots
#                              for lecturer in permanent_lectures + non_permanent_lectures
#                              if (classroom, time_slot, day) in x.keys()]
            
#             # Add constraint using pulp.lpSum
#             lp_problem += pulp.lpSum(decision_vars) == 1 

# # Constraint: Ensure CTJ1A2 course is studied by group 4 and group 5 and is only studied once a week
# for subject in subjects:
#     if subject in ['CRI1A2', 'CRI1A2']:
#         for group in ['SE-47-01', 'SE-47-02', 'SE-47-03', 'SE-47-04']:
#             for day in days:
#                 # Create a list of decision variables for the subject, group, and day
#                 decision_vars = [x[classroom][group][time_slot][day][subject][lecturer]
#                                  for classroom in rooms
#                                  for time_slot in timeslots
#                                  for lecturer in permanent_lectures + non_permanent_lectures
#                                  if (classroom, time_slot, day) in x.keys()]
                
#                 # Add constraint using pulp.lpSum
#                 lp_problem += pulp.lpSum(decision_vars) == 1  # CTJ1A2 is studied only once a week by each group


# # Constraint: Ensure CTJ1A2 course is studied by group 4 and group 5 and is only studied once a week
# for subject in subjects:
#     if subject == 'CSI1A2':
#         for group in ['DS-47-01', 'DS-47-02', 'DS-47-03']:
#             for day in days:
#                 # Create a list of decision variables for the subject, group, and day
#                 decision_vars = [x[classroom][group][time_slot][day][subject][lecturer]
#                                  for classroom in rooms
#                                  for time_slot in timeslots
#                                  for lecturer in permanent_lectures + non_permanent_lectures
#                                  if (classroom, time_slot, day) in x.keys()]
                
#                 # Add constraint using pulp.lpSum
#                 lp_problem += pulp.lpSum(decision_vars) == 1  # CTJ1A2 is studied only once a week by each group





In [14]:
# Solve the ILP problem

lp_problem.solve()
end_time = time.time()

# Print the status of the solution
print("Status:", pulp.LpStatus[lp_problem.status])

Status: Infeasible


In [15]:
# Record resource usage after solving
monitor_resource_usage_after()

### Result

In [16]:
# Initialize timetable dictionary
timetable = {classroom: {day: {timeslot: None for timeslot in timeslots} for day in days} for classroom in rooms}

# Extract the schedule from the ILP solution
for classroom in rooms:
    for group in students_groups:
        for time_slot in timeslots:
            for day in days:
                for subject in subjects:
                    for lecturer in permanent_lectures + non_permanent_lectures:
                        if pulp.value(x[classroom][group][time_slot][day][subject][lecturer]) == 1:
                            timetable[classroom][day][timeslot] = (subject, group, lecturer)

# Print the timetable
for classroom in rooms:
    print(f"Timetable for {classroom}:")
    for day in days:
        print(f"  {day}:")
        for timeslot in timeslots:
            entry = timetable[classroom][day][time_slot]
            if entry:
                course, group, lecturer = entry
                print(f"    {timeslot}: {course} for {group} with {lecturer}")
            else:
                print(f"    {timeslot}: Free")
    print()

Timetable for TULT-0601:
  Monday:
    timeslot1: Free
    timeslot2: Free
    timeslot3: Free
    timeslot4: Free
    timeslot5: Free
    timeslot6: Free
    timeslot7: Free
    timeslot8: Free
    timeslot9: Free
    timeslot10: Free
    timeslot11: Free
    timeslot12: Free
  Tuesday:
    timeslot1: Free
    timeslot2: Free
    timeslot3: Free
    timeslot4: Free
    timeslot5: Free
    timeslot6: Free
    timeslot7: Free
    timeslot8: Free
    timeslot9: Free
    timeslot10: Free
    timeslot11: Free
    timeslot12: Free
  Wednesday:
    timeslot1: Free
    timeslot2: Free
    timeslot3: Free
    timeslot4: Free
    timeslot5: Free
    timeslot6: Free
    timeslot7: Free
    timeslot8: Free
    timeslot9: Free
    timeslot10: Free
    timeslot11: Free
    timeslot12: Free
  Thursday:
    timeslot1: Free
    timeslot2: Free
    timeslot3: Free
    timeslot4: Free
    timeslot5: Free
    timeslot6: Free
    timeslot7: Free
    timeslot8: Free
    timeslot9: Free
    timeslot10: Free

In [21]:
# Initialize a dictionary to hold the timetable for each room
timetable = {room: pd.DataFrame('', index=pd.MultiIndex.from_product([days, timeslots], names=['Day', 'Timeslot']), columns=['Group', 'Subject', 'Lecturer']) for room in rooms}

# Fill the timetable with the schedule
for classroom in rooms:
    for group in students_groups:
        for time_slot in timeslots:
            for day in days:
                for subject in subjects:
                    for lecturer in non_permanent_lectures:
                        if pulp.value(x[classroom][group][time_slot][day][subject][lecturer]) == 1:
                            timetable[classroom].loc[(day, time_slot)] = [group, subject, lecturer]

# Function to print the timetable for each room in a more readable format
def print_timetable(timetable):
    for room, schedule in timetable.items():
        print(f"\nTimetable for {room}:\n")
        print(schedule.to_string())  # Using .to_string() to print the entire DataFrame

# Print the timetable for each room
print_timetable(timetable)


Timetable for TULT-0601:

                         Group Subject Lecturer
Day       Timeslot                             
Monday    timeslot1                            
          timeslot2                            
          timeslot3   IF-47-03  CII1A3      CDF
          timeslot4                            
          timeslot5                            
          timeslot6                            
          timeslot7                            
          timeslot8                            
          timeslot9                            
          timeslot10  IF-47-06  CII1A3      APY
          timeslot11  IT-47-04  CII1A3      DVI
          timeslot12                           
Tuesday   timeslot1                            
          timeslot2                            
          timeslot3                            
          timeslot4                            
          timeslot5                            
          timeslot6                            
          tim

In [18]:
print("Room assignment:")
for classroom in rooms:
    for group in students_groups:
        for time_slot in timeslots:
            for day in days:
                for subject in subjects:
                    for lecturer in permanent_lectures + non_permanent_lectures:
                        if pulp.value(x[classroom][group][time_slot][day][subject][lecturer]) == 1:
                            print(f"Room: {classroom}, Group: {group}, Timeslot: {time_slot}, Day: {day}, Course: {subject}, Lecture: {lecturer}")

Room assignment:
Room: TULT-0601, Group: IF-47-02, Timeslot: timeslot11, Day: Friday, Course: CII1A3, Lecture: CDF
Room: TULT-0601, Group: IF-47-03, Timeslot: timeslot3, Day: Monday, Course: CII1A3, Lecture: CDF
Room: TULT-0601, Group: IF-47-04, Timeslot: timeslot12, Day: Tuesday, Course: CII1A3, Lecture: CDF
Room: TULT-0601, Group: IF-47-06, Timeslot: timeslot10, Day: Monday, Course: CII1A3, Lecture: APY
Room: TULT-0601, Group: IF-47-07, Timeslot: timeslot2, Day: Thursday, Course: CII1A3, Lecture: APY
Room: TULT-0601, Group: IF-47-08, Timeslot: timeslot2, Day: Wednesday, Course: CRI1A2, Lecture: DOK
Room: TULT-0601, Group: IF-47-09, Timeslot: timeslot5, Day: Thursday, Course: CII1A3, Lecture: DRI
Room: TULT-0601, Group: IF-47-11, Timeslot: timeslot5, Day: Wednesday, Course: CRI1A2, Lecture: FAZ
Room: TULT-0601, Group: IT-47-03, Timeslot: timeslot7, Day: Tuesday, Course: CII1A3, Lecture: SBG
Room: TULT-0601, Group: IT-47-03, Timeslot: timeslot8, Day: Thursday, Course: CII1A3, Lecture: 

In [19]:
# Print resource usage statistics
print(f"Time taken to solve: {end_time - start_time:.2f} seconds")
print(f"CPU usage before: {cpu_usage[0]:.2f}%")
print(f"CPU usage after: {cpu_usage[1]:.2f}%")
print(f"Memory usage before: {memory_usage_before[0]:.2f} MB")
print(f"Memory usage after: {memory_usage_after[0]:.2f} MB")
print(f"Disk usage before: {disk_usage_before[0]:.2f} MB")
print(f"Disk usage after: {disk_usage_after[0]:.2f} MB")

Time taken to solve: 5564.24 seconds
CPU usage before: 1.90%
CPU usage after: 14.30%
Memory usage before: 14554.88 MB
Memory usage after: 26594.65 MB
Disk usage before: 292574.98 MB
Disk usage after: 292574.98 MB


In [20]:
# Create a dictionary of DataFrames with room names as keys
room_sheets = {}
for room in rooms:  # Assuming classrooms is the list of room names
    # Filter decision variables for the current room
    room_decisions = [(var.name.split("_")[1], var.name.split("_")[2], var.name.split("_")[3], var.name.split("_")[4], var.name.split("_")[5]) for var in lp_problem.variables() if var.name.split("_")[2] == room]

    # Create a DataFrame from the filtered decision variables
    room_df = pd.DataFrame(room_decisions, columns=['Day', 'Time Slot', 'Subject', 'Lecturer', 'Group'])

    # Create a custom order for time slots
    time_slot_order = ['timeslot1', 'timeslot2', 'timeslot3', 'timeslot4', 'timeslot5', 'timeslot6', 'timeslot7', 'timeslot8', 'timeslot9', 'timeslot10', 'timeslot11', 'timeslot12']

    # Create a custom order for days
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

    # Pivot the DataFrame to get days as columns and time slots as rows
    pivoted_df = room_df.pivot(index='Time Slot', columns='Day', values=['Subject', 'Group', 'Lecturer'])

    # Reorder the index based on custom order for time slots
    pivoted_df = pivoted_df.reindex(index=time_slot_order)

    # Reorder the columns based on custom order for days
    pivoted_df = pivoted_df.reindex(columns=day_order, level=1)

    # Add the pivoted DataFrame to the dictionary
    room_sheets[room] = pivoted_df

# Write each DataFrame to a separate Excel sheet
with pd.ExcelWriter('Hasil/room_schedule.xlsx') as writer:
    for room, df in room_sheets.items():
        df.to_excel(writer, sheet_name=room)


FileNotFoundError: [Errno 2] No such file or directory: 'Hasil/room_schedule.xlsx'